# Introduction


This is a database of the various measurements for benthic sediment, waterTidalBenthic quality, and biomass in the Chesapeake Bay. Due to the organization of the data, the measured parameters are in a column. In the [Preparing to combine datasets section](#preparing-to-combine-datasets) we will [turn parameters into columns](#turn-parameters-into-columns).

In [1]:
import pandas as pd
import numpy as np
import json


The CSVs for both the Tidal Benthic database and thhe Water Quakity database have the following columns. When different names are used for the same information across different datasets, all names are included. These differences will be handked in cleaning.

Here is a descriptor of the columns, from [The 2012 Users Guide to CBP Biological Monitoring Data](https://d18lev1ok5leia.cloudfront.net/chesapeakebay/documents/guide2012_final.pdf) and [Guide to Using Chesapeake Bay Program Water Quality Monitoring Data](https://d18lev1ok5leia.cloudfront.net/chesapeakebay/documents/wq_data_userguide_10feb12_mod.pdf):
- `CBSeg2003` 2003 Chesapeake Bay Segment Designation. Divided into regions based on circulation and salinity properties. We used 8 from the Bay proper, 2 adjoining Bays, and 1 adjoining sound.
- `CBSeg2003Description` 2003 Chesapeake Bay Segment Designation Description in the format Location-Salinity. The locations are Chesapeake Bay, Eastern Bay, Mobjack Bay, and Tangier Sound. The salinity levels are tidal fresh (0.0 - 0.5 parts per thousand),
oligohaline (0.5 - 5.0 parts per thousand), mesohaline (5.0 - 18.0 parts per thousand), and polyhaline (greater than 18.0 parts per thousand). 
- `Station` the sampling station
- `TotalDepth` is Total *Station* Depth (Meters)
- `Source` the s the agency or company that collected the water quality samples or took the field measurements
- `Latitude` and  `Longitude`, the Latitude and Longitude for the sampling station
- `FieldActivityId` (tidal  benthic database) and `EventId` (Water Quality database), are Database Generated Event Identification Numbers, mostly added from the monitoring event data.
- `EventId` (sediment, water, taxonomic) and `BiologicalEventId` (biomass, ibi) encode both the `FieldActivityId` and sampling information. The same time, date, location will have one `FieldActivityId` but possibly multpile `EventId` or `BiologicalEventId`.
- `SampleDate` Sampling date (MM/DD/YYYY). 
- `SampleTime` Sample Collection Time (HH:MM)
- `Layer` Layer of water column in which sample was taken. However, this column is not consistently coded in the taxonomic counts data.
- `SampleReplicate` This parameter combines the sample replicate number with a sample type
descriptor. 
     - S1, Sample 1. The vast majority of the data.
     - S2, Sample 2 
- `ReportingParameter` (sediment), `ReportedParameter` (water quality), `IBIParameter` (biomass, indicator of benthic integrity), `Parameter` (Water Quality database) Sampling Parameter.
- `ReportingValue` (taxonomic), `ReportedValue` (sediment, water quality), `IBIValue` (biomass, indicator of benthic integrity), `MeasureValue` (Water Quality database) the value of the parameter.
- `ReportingUnits` (sediment, taxonomic), `ReportedUnits` (water quality), `Unit` (Water Quality Database). This parameter describes the units in which a substance is measured. 
- `ProjectIdentifier` (Tidal Benthic Database) or `Project` (Water Quality database), abbreviation for the project that collected the data.


The Tidal Benthic data also has:
- `Units` units for the sample volume. Always centimeters (cubic centimeters?).
- `SampleVolume` Total Volume of Sample
- `PDepth`, Composite Sample Cut Off Depth (meters)
- `Salzone`, Salinity Zone
     - HM, High Mesohaline =>12 TO 18 parts per thousand
     - LM, Low Mesohaline =>5.0 TO 12 parts per thousand
     - M, Mesohaline =>5.0 TO 18 parts per thousand
     - O, Oligohaline =>0.5 TO 5.0 parts per thousand
     - P, Polyhaline =>18 parts per thousand
     - TF, Tidal Fresh < 0.5 parts per thousand

Additionally, the taxonomic dataset has 
- `GMethod` Chesapeake Bay Program Gear Method Code. Codes represent information relating to the type of field gear used to collect samples for all analysis. In this dataset all are 7, Plankton Pump
- `TSN` ITIS Taxon Serial Number, unique to the species. When used in conjunction with the NODC, the TSN
overcomes the problem of numeric changes in the NODC code whenever species are reclassified. 
- `LatinName` Species Latin Name 
- `Size` Cell Size Groupings when taken. Some species have different measurements for different sizes. 
- `LifeStageDescription`, a numeric code of the life stage. Most are 89 - not specified.

The Water Quality database also has:
- `Cruise`, This alpha-numeric code identifies the cruise to which the data observation belongs. Cruise identification
is useful for grouping data that are collected over a range of sample dates, but that are considered data
for a specific sampling period. (Should be in both datasets)
- `Program`, The PROGRAM code was added to the database design because Maryland DNR has adopted a
project-oriented approach to water quality data management. This approach relies upon the use
of PROGRAM (WQMP at DNR) and PROJECT (MAIN and TRIB) codes.
- `Agency`, agencies that are ultimately responsible for ensuring the proper processing, storage, and submission or serving of water quality data
- `UpperPycnocline`, Depth of upper pycnocline
- `LowerPycnocline`, Depth of lower pycnocline
- `Depth`
- `Method`
- `Lab`
- `Problem`
- `Details`
- `TierLevel`
- Weather information. The other than air temperature, these are encoded with the description, but most already have a numeric code in the [Guide to Using Chesapeake Bay Program Water Quality Monitoring Data](https://d18lev1ok5leia.cloudfront.net/chesapeakebay/documents/wq_data_userguide_10feb12_mod.pdf)
     - `AirTemp` in degrees Celcius
     - `WindSpeed` the best estimate of the wind speed experienced during a sampling event, in a 10 knot range. Numerical dictionary available.
     - `WindDirection` the prevailing wind direction experienced during a sampling event. 
     - `PrecipType` the type of precipitation experienced during a sampling event. Numerical dictionary available.
     - `TideStage` the tidal stage experienced during a sampling event.
     - `WaveHeight` the best estimate of the wave conditions experienced during a sampling event, as a range. Numerical dictionary available.
     - `CloudCover` best estimate of the percent cloud cover experienced during a sampling event. Numerical dictionary available.

# Cleaning Individual Datasets

The first step of data cleaning is working with the individual datasets and determining what information we need to keep. Some of this cleaning will be done with the aid of the DataWrangler extension in VSCode. Each of the datasets seem to have only one value in the `Layer` column, but this maybe useful in combining with other datasets. Sediment, BioMass, WaterQuality and Indicator of Benthic Integrity all have "B" for Bottom or no value, while Taxonomic counts all has "D," which is not in the userguide.

## Common Functions

For each of the datasets, we will read in the CSV, replace and empty values and NaN with `None`.  We also settle on a consistent naming scheme for reported parameter, value, and units. For taxonomic data, this will largely be handled with later cleaning.

For the`Sediment and WaterQuality datasets, we will create a dictionary between the parameter, its units, and its meaning. Since BioMass and Indicator of Benthic Integrity have 126 unique parameters and no corresponding units column, and the dictionaries must be hand coded later.

In [2]:
def read_and_intial_clean(file_path):
    df = pd.read_csv(file_path)

    # Rename Units to be more clear
    df = df.rename(columns={'Units' : 'SampleVolumeUnits'})
    
    # rename reported parameter, value, and units
    if 'ReportingParameter' in df.columns:
        df = df.rename(columns={'ReportingParameter': 'ReportedParameter'})
    if 'IBIParameter' in df.columns:
        df = df.rename(columns={'IBIParameter': 'ReportedParameter'})
    if 'Parameter' in df.columns:
        df = df.rename(columns={'Parameter': 'ReportedParameter'})
    if 'ReportingValue' in df.columns:
        df = df.rename(columns={'ReportingValue': 'ReportedValue'})
    if 'IBIValue' in df.columns:
        df = df.rename(columns={'IBIValue': 'ReportedValue'})
    if 'MeasureValue' in df.columns:
        df = df.rename(columns={'MeasureValue': 'ReportedValue'})
    if 'ReportingUnits' in df.columns:
        df = df.rename(columns={'ReportingUnits': 'ReportedUnits'})
    if 'Unit' in df.columns:
        df = df.rename(columns={'Unit': 'ReportedUnits'})
    if 'Units' in df.columns:
        df = df.rename(columns={'Units': 'SampleVolumeUnits'})
    if 'Project' in df.columns:
        df = df.rename(columns={'Project': 'ProjectIdentifier'})
        

    df_clean = df.replace('', np.nan).where(df.notna(), None)
    return df_clean

def param_dict(df, meanings_list):
    parameters = df['ReportedParameter'].unique()

    # Meanings list must be carefully created from the userguide
    # Create a mapping from parameters to their meanings
    param_meaning = {param: meaning for param, meaning in zip(parameters, meanings_list)}

    # Create the initial dictionary with units and empty types
    param_dict = {param: {'Meaning': "", 'Units': unit} for param, unit in zip(df['ReportedParameter'], df['ReportedUnits'])}

    # Update the dictionary with the meanings
    for param in param_dict:
        if param in param_meaning:
            param_dict[param]['Meaning'] = param_meaning[param]

    return param_dict


We will also want a function to check if two columns encode the same information.

In [3]:
def compare_column_values(df,column1,column2):
    # Group by column1 and check the unique values in column2
    groups = df.groupby(column1)[column2].nunique()

    # Identify column1 values where LifeStageDescriptions has more than one unique value
    disagreeing_a_values = groups[groups > 1].index

    # Filter the DataFrame
    filtered_df = df[df[column1].isin(disagreeing_a_values)]

    filtered_df[[column1,column2]].drop_duplicates().sort_values(by=column1)
    return filtered_df

## Tidal Benthic Dataset

### Sediment and water quality cleaning

The sediment and water quality datasets have similar structures. It appears these are actually the same datasets. We will create dictionaries for both, as well.

In [122]:
sediment_file = '../../data/plankton-patrolCSVs/plank_ChesapeakeBenthicSediment.csv'
waterTidalBenthic_file = '../../data/plankton-patrolCSVs/plank_ChesapeakeBenthicWaterQuality.csv'

sediment = read_and_intial_clean(sediment_file)
waterTidalBenthic = read_and_intial_clean(waterTidalBenthic_file)

In [123]:
print("Sediment parameters: ", sediment['ReportedParameter'].unique())
print("WaterQuality parameters: ", waterTidalBenthic['ReportedParameter'].unique())

Sediment parameters:  ['MOIST' 'SAND' 'TC' 'TIC' 'SILTCLAY' 'TOC' 'TN' 'VOLORG' 'KURTOSIS'
 'CLAY' 'MEANDIAM' 'SORT' 'SKEWNESS']
WaterQuality parameters:  ['PH' 'WTEMP' 'DO' 'DO_SAT_P' 'SALINITY' 'SPCOND']


Since three of the parameters in the sediment dataset are statistical measures (kurtosis, skewness, sort), we need to remove these rows.

In [124]:
# List of parameters to remove
parameters_to_remove = ['KURTOSIS', 'SKEWNESS', 'SORT']

# Remove rows where 'ReportedParameter' is in the list of parameters to remove
sediment = sediment[~sediment['ReportedParameter'].isin(parameters_to_remove)]

print("Sediment parameters: ", sediment['ReportedParameter'].unique())

Sediment parameters:  ['MOIST' 'SAND' 'TC' 'TIC' 'SILTCLAY' 'TOC' 'TN' 'VOLORG' 'CLAY'
 'MEANDIAM']


Now consulting [2012 Users Guide to CBP Biological Monitoring Data](https://d18lev1ok5leia.cloudfront.net/chesapeakebay/documents/guide2012_final.pdf) and keeping the information in the same order, we get

In [125]:
sediment_parameters_meanings = ['Sediment Moisture Percentage', 'Sand Content, Percent', 'Total Carbon Content','Total Inorganic Carbonate Content','Silt Clay Content, Percent','Total Organic Carbon','Total Nitrogen','Volatile Organic, Percent','Clay Content, Percent','Mean Sediment Diameter']

waterTidalBenthic_parameters_meanings = ['pH', 'Water Temperature', 'Dissolved Oxygen', 'Dissolved oxygen relative to theoretical value at saturation', 'Salinity', 'Specific Conductance At 25 C']

In [126]:
sediment_param_dict = param_dict(sediment,sediment_parameters_meanings)
waterTidalBenthic_param_dict = param_dict(waterTidalBenthic,waterTidalBenthic_parameters_meanings)

Now that we have our dictionary, we can drop the `ReportedUnits` column.

In [127]:
sediment = sediment.drop(columns='ReportedUnits')
waterTidalBenthic = waterTidalBenthic.drop(columns='ReportedUnits')

In [128]:
sediment_columns = sediment.columns
waterTidalBenthic_columns = waterTidalBenthic.columns

# Find common columns
sediment_and_water_columns = list(set(sediment_columns) & set(waterTidalBenthic_columns))

# Find columns only in biomass
columns_only_in_sediment = list(set(sediment_columns) - set(waterTidalBenthic_columns))

# Find columns only in ibi
columns_only_in_water = list(set(waterTidalBenthic_columns) - set(sediment_columns))

print("Columns in sediment and water quality: ", sediment_and_water_columns)
print("Columns only in Sediment: ", columns_only_in_sediment)
print("Columns only in Water Quality: ", columns_only_in_water)

Columns in sediment and water quality:  ['Salzone', 'SampleDate', 'Latitude', 'FieldActivityId', 'SampleTime', 'Layer', 'Longitude', 'SampleVolume', 'ProjectIdentifier', 'EventId', 'ReportedParameter', 'ReportedValue', 'Source', 'SampleReplicate', 'TotalDepth', 'Station', 'CBSeg2003Description', 'SampleVolumeUnits', 'CBSeg2003', 'PDepth']
Columns only in Sediment:  []
Columns only in Water Quality:  ['SampleDepth', 'WQMethod']


Since  `WQMethod` is about the method used to collect the sample, we can drop it. We also need to drop `EventId` to prevent issues merging (since it also track the parameter).

In [129]:
sediment = sediment.drop(columns=['EventId'])
waterTidalBenthic = waterTidalBenthic.drop(columns=['WQMethod','EventId'])

### BioMass and Indicator of Benthic Integrity cleaning

Since the BioMass and Indicator of Benthic Integrity datasets have similar structures, we will handle them together. We will then check if there are any differences between the two datasets.

In [78]:
biomass_file = '../../data/plankton-patrolCSVs/plank_ChesapeakeBenthicBioMass.csv'
ibi_file = '../../data/plankton-patrolCSVs/plank_ChesapeakeBenthicIBI.csv'


biomass = read_and_intial_clean(biomass_file)
ibi = read_and_intial_clean(ibi_file)

In [79]:
# Merge with indicator
merged_df = biomass.merge(ibi, how='outer', indicator=True)

# Rows only in biomass
only_in_biomass = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])

# Rows only in ibi
only_in_ibi = merged_df[merged_df['_merge'] == 'right_only'].drop(columns=['_merge'])

print("Rows only in biomass:")
print(only_in_biomass)

print("Rows only in ibi:")
print(only_in_ibi)

Rows only in biomass:
Empty DataFrame
Columns: [CBSeg2003, CBSeg2003Description, Station, Latitude, Longitude, FieldActivityId, SampleDate, SampleTime, Layer, TotalDepth, BiologicalEventId, Source, SampleReplicate, ReportedParameter, ReportedValue, ProjectIdentifier, SampleVolumeUnits, SampleVolume, PDepth, Salzone]
Index: []
Rows only in ibi:
Empty DataFrame
Columns: [CBSeg2003, CBSeg2003Description, Station, Latitude, Longitude, FieldActivityId, SampleDate, SampleTime, Layer, TotalDepth, BiologicalEventId, Source, SampleReplicate, ReportedParameter, ReportedValue, ProjectIdentifier, SampleVolumeUnits, SampleVolume, PDepth, Salzone]
Index: []


Since there are the same dataset, we will only work with biomass. Let's at least find out how many parameters we have and what columns we have.

In [80]:
biomass_parameters = biomass['ReportedParameter'].unique()
biomass_columns = biomass.columns


print("BioMass paramenters: ", len(biomass_parameters))
print("Biomass columns: ", biomass_columns)

BioMass paramenters:  126
Biomass columns:  Index(['CBSeg2003', 'CBSeg2003Description', 'Station', 'Latitude', 'Longitude',
       'FieldActivityId', 'SampleDate', 'SampleTime', 'Layer', 'TotalDepth',
       'BiologicalEventId', 'Source', 'SampleReplicate', 'ReportedParameter',
       'ReportedValue', 'ProjectIdentifier', 'SampleVolumeUnits',
       'SampleVolume', 'PDepth', 'Salzone'],
      dtype='object')


Finally, are `BiologicalEventId` and `FieldActivityId` encoding the same information?

In [81]:
compare_column_values(biomass,'FieldActivityId','BiologicalEventId')

,CBSeg2003,CBSeg2003Description,Station,Latitude,Longitude,FieldActivityId,SampleDate,SampleTime,Layer,TotalDepth,BiologicalEventId,Source,SampleReplicate,ReportedParameter,ReportedValue,ProjectIdentifier,SampleVolumeUnits,SampleVolume,PDepth,Salzone
10555,CB4MH,Chesapeake Bay-Mesohaline Region,001,38.4193,-76.41889,218369,9/19/2006,13:40:00,None,1.9,70216,VERSAR/EME/BEL,S1,GRAND_SCORE,1.56,BEN,None,NaN,NaN,None
10556,CB4MH,Chesapeake Bay-Mesohaline Region,001,38.4193,-76.41889,218369,9/19/2006,13:40:00,None,1.9,70216,VERSAR/EME/BEL,S1,PCT_CARN_OMN,6.90,BEN,None,NaN,NaN,None
10557,CB4MH,Chesapeake Bay-Mesohaline Region,001,38.4193,-76.41889,218369,9/19/2006,13:40:00,None,1.9,70216,VERSAR/EME/BEL,S1,PCT_CARN_OMN,11.76,BEN,None,NaN,NaN,None
10558,CB4MH,Chesapeake Bay-Mesohaline Region,001,38.4193,-76.41889,218369,9/19/2006,13:40:00,None,1.9,70216,VERSAR/EME/BEL,S1,PCT_CARN_OMN,15.38,BEN,None,NaN,NaN,None
10559,CB4MH,Chesapeake Bay-Mesohaline Region,001,38.4193,-76.41889,218369,9/19/2006,13:40:00,None,1.9,70216,VERSAR/EME/BEL,S1,PCT_PI_ABUND,41.18,BEN,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20802,CB4MH,Chesapeake Bay-Mesohaline Region,001,38.4191,-76.41850,224939,9/13/2012,12:18:00,None,2.2,73162,VERSAR/EME/BEL,S2,TRICHOPTERA_TAXA_CNT,3.00,BEN,None,NaN,NaN,None
20803,CB4MH,Chesapeake Bay-Mesohaline Region,001,38.4191,-76.41850,224939,9/13/2012,12:18:00,None,2.2,73162,VERSAR/EME/BEL,S2,TRICHOPTERA_TAXA_CNT_R,3.00,BEN,None,NaN,NaN,None
20804,CB4MH,Chesapeake Bay-Mesohaline Region,001,38.4191,-76.41850,224939,9/13/2012,12:18:00,None,2.2,73162,VERSAR/EME/BEL,S2,TRICHOPTERA_TAXA_COUNT_NO_HYDR,2.00,BEN,None,NaN,NaN,None
20805,CB4MH,Chesapeake Bay-Mesohaline Region,001,38.4191,-76.41850,224939,9/13/2012,12:18:00,None,2.2,73162,VERSAR/EME/BEL,S2,TRICHOPTERA_TAXA_COUNT_NO_HYDR_R,2.00,BEN,None,NaN,NaN,None


We also need to drop `BiologicalEventId` to prevent issues merging (since it also track the parameter).

In [130]:
biomass = biomass.drop(columns='BiologicalEventId')

### Taxonomic Counts

The taxonomic counts dataset is structured fairly differently than the others. This is partly due to encoding taxonomic data, although not entirely. For example, there are two for encoding taxonomic information; one is numeric and the other is Latin name.

In [82]:
taxonomic_file = '../../data/plankton-patrolCSVs/plank_ChesapeakeBenthicTaxonomic.csv'

taxonomic = read_and_intial_clean(taxonomic_file)

Let's create a dictionary for `LatinName` and the corresponding `TSN` number

In [83]:
taxonomic_names_dict = {param: meaning for param, meaning in zip(taxonomic['TSN'], taxonomic['LatinName'])}

Let's also create a dictionary for `LifeStageDescription`, from the table in the user guide.

In [84]:
taxonomic['LifeStageDescription'].unique()

array([ 89.,  79., 248.,  76.,  97., 247., 245.,  53.,  21., 225.,  nan,
        93.,  52., 232.])

In [85]:
life_stage_dict = {
    89.: 'Not Specified',  
    79.: 'Species', 
    248.: 'Immature Without Cap. Chaete',  
    76.: 'Group',  
    97.: 'Larvae', 
    247.: 'Immature With Cap. Chaete', 
    245.: 'Type',  
    53.: 'Species B',  
    21.: 'Pupae', 
    225.: 'Complex',  
    93.: 'Juvenile',
    52.: 'Species A', 
    232.: 'Species M'
}

Since it is not clear how best to combine the numeric informatin in `TSN` and `LifeStageDescription` (life stages as a decimal, maybe?), we will combine the `LatinName` and `LifestageDescription`. Let's replace the `LifeStateDescription` with the actual description.

In [86]:
taxonomic['LifeStageDescription'] = taxonomic['LifeStageDescription'].replace(life_stage_dict)

Let's see how many `LatinNames` are measured at multiple life stages.

In [87]:
# Group by LatinName and check the unique values in LifeStageDescriptions
groups = taxonomic.groupby('LatinName')['LifeStageDescription'].nunique()

# Identify LatinName values where LifeStageDescriptions has more than one unique value
disagreeing_a_values = groups[groups > 1].index

# Filter the DataFrame
filtered_df = taxonomic[taxonomic['LatinName'].isin(disagreeing_a_values)]

filtered_df[['LatinName','LifeStageDescription']].drop_duplicates().sort_values(by='LatinName')

,LatinName,LifeStageDescription
24636,Ampelisca,Species
8691,Ampelisca,Juvenile
46,Axarus,Not Specified
641,Axarus,Species
11432,Bivalvia,Species
11076,Bivalvia,Species B
33,Chironomidae,Larvae
351,Chironomidae,Pupae
629,Chironomidae,Not Specified
25684,Enchytraeidae,Not Specified


We combine the `LatinName` and `LifeStageDescription` columns to create the `ReportedParameter` column. We will drop `Not Specified`.

In [88]:
taxonomic = taxonomic.copy()

taxonomic['ReportedParameter'] = taxonomic['LatinName'] + ' ' +taxonomic['LifeStageDescription'].replace('Not Specified', '')


Is `TSN` always the same for `LatinName`? 

In [89]:
# Group by LatinName and check the unique values in TSN
groups = taxonomic.groupby('LatinName')['TSN'].nunique()

# Identify LatinName values where EventId has more than one unique value
disagreeing_a_values = groups[groups > 1].index

# Filter the DataFrame
taxonomic[taxonomic['TSN'].isin(disagreeing_a_values)].drop_duplicates

<bound method DataFrame.drop_duplicates of Empty DataFrame
Columns: [CBSeg2003, CBSeg2003Description, Station, Latitude, Longitude, SampleType, FieldActivityId, SampleDate, SampleTime, Layer, TotalDepth, ReportedValue, ReportingUnit, EventId, Source, GMethod, TSN, LifeStageDescription, LatinName, ProjectIdentifier, SampleVolumeUnits, Salzone, PDepth, SampleVolume, ReportedParameter]
Index: []

[0 rows x 25 columns]>

Just to be safe, we will keep the `TSN` column and drop the `LatinName` column, since `TSN` is already a numeric value. We will keep the `ReportedParameter` column and the `LifeStage` column, also to be safe.

Also, for some reason every `ReportedUnit` is CM. The fact that there are many rows that only differ by the reported value suggest some are counts (and some are something else?). These values also vary a lot in scale, so the unit seems to not actually be consistent. We will drop this column.

In [90]:
taxonomic_clean = taxonomic.drop(columns=['LatinName','ReportingUnit'])

In [91]:
taxonomic_columns = taxonomic_clean.columns
print("Taxonomic columns: ",taxonomic_columns)

Taxonomic columns:  Index(['CBSeg2003', 'CBSeg2003Description', 'Station', 'Latitude', 'Longitude',
       'SampleType', 'FieldActivityId', 'SampleDate', 'SampleTime', 'Layer',
       'TotalDepth', 'ReportedValue', 'EventId', 'Source', 'GMethod', 'TSN',
       'LifeStageDescription', 'ProjectIdentifier', 'SampleVolumeUnits',
       'Salzone', 'PDepth', 'SampleVolume', 'ReportedParameter'],
      dtype='object')


#### Dealing with ReportedValues column -- not used later

Since some values in the `ReportedValue` column appear to be counts and some are size, let's see if there are any rows that agree in all columns except `ReportedValue`.

In [92]:
# Columns to exclude from comparison
exclude_columns = ['ReportedValue']

# Columns to compare
compare_columns = [col for col in taxonomic_clean.columns if col not in exclude_columns]

# Group by the columns to compare
grouped = taxonomic_clean.groupby(compare_columns)

# Initialize a dictionary to collect rows based on group size
group_dict = {
    2: [],
    3: [],
    'more_than_3': []
}

# Iterate through each group
for group_key, group in grouped:
    num_rows = len(group)
    
    # Separate groups based on number of rows
    if num_rows == 2:
        group_dict[2].append(group)
    elif num_rows == 3:
        group_dict[3].append(group)
    elif num_rows > 3:
        group_dict['more_than_3'].append(group)

# Function to create DataFrame from groups
def create_dataframe(groups, sort_by):
    return pd.concat(groups).sort_values(by=sort_by) if groups else pd.DataFrame()

# Create DataFrames for viewing
df_groups_with_2_rows = create_dataframe(group_dict[2], ['ReportedParameter', 'EventId'])
df_groups_with_3_rows = create_dataframe(group_dict[3], ['ReportedParameter', 'EventId', 'ReportedValue'])
df_groups_with_more_than_3_rows = create_dataframe(group_dict['more_than_3'], ['ReportedParameter', 'EventId', 'ReportedValue']).drop_duplicates()

# Print shapes
print("Two entries=: ",df_groups_with_2_rows.shape)
print("Three entries=: ",df_groups_with_3_rows.shape)
print("More than three entries=: ",df_groups_with_more_than_3_rows.shape)

Two entries=:  (20484, 23)
Three entries=:  (1182, 23)
More than three entries=:  (4090, 23)


## Water Quality Dataset

In [47]:
water_file = '../../data/plankton-patrolCSVs/plank_ChesapeakeWaterQuality.csv'
water = read_and_intial_clean(water_file)

/var/folders/rl/kqt6tbv90l9_pwc4927vdb340000gn/T/ipykernel_34486/404319392.py:2: DtypeWarning: Columns (2,24,25,29,31,32,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Let's investigate the DtypeWarning.

In [48]:
water.columns[[24, 29, 33]]

Index(['BiasPC', 'FieldActivityEventType', 'PrecipType'], dtype='object')

`BiasPC` and `FieldActivityEventType` are almost entirely missing, so we can drop those. Let's look at the unique entries in `PrecipType`.

In [49]:
water['PrecipType'].unique()

array([None, 'Drizzle', 'Rain', 'Squally', 'Heavy Rain',
       'Frozen Precipitation'], dtype=object)

In [50]:
water = water.drop(columns=['BiasPC','FieldActivityEventType'])

Now we can do some further renaming to match with the Tidal Bethic datasets. Since `EventId` is used differently in the two datasets, let's rename this on `FieldActivityId`.

In [51]:
water = water.rename(columns={'EventId': 'FieldActivityId'})

In [52]:
water.columns

Index(['CBSeg2003', 'FieldActivityId', 'Cruise', 'Program',
       'ProjectIdentifier', 'Agency', 'Source', 'Station', 'SampleDate',
       'SampleTime', 'TotalDepth', 'UpperPycnocline', 'LowerPycnocline',
       'Depth', 'Layer', 'SampleType', 'SampleReplicateType',
       'ReportedParameter', 'Qualifier', 'ReportedValue', 'ReportedUnits',
       'Method', 'Lab', 'Problem', 'Details', 'Latitude', 'Longitude',
       'TierLevel', 'AirTemp', 'WindSpeed', 'WindDirection', 'PrecipType',
       'TideStage', 'WaveHeight', 'CloudCover'],
      dtype='object')

We also remove the columns that correspond to how the data was collected or measured (`Program`, `Agency`, `Method`, `Lab`, `TierLevel`). We also remove `SampleReplicateType`, as this could cause errors with merging, with different parts of the sample being in different replicates.

In [53]:
water = water.drop(columns=['Program','Agency','Method', 'Lab','TierLevel','SampleReplicateType'])

Now we create our parameter dictionary.

In [54]:
print("Water parameters: ", water['ReportedParameter'].unique())

Water parameters:  ['CHLA' 'DIN' 'DO' 'DON' 'DOP' 'KD' 'NH4F' 'NO23F' 'NO2F' 'NO3F' 'PC' 'PH'
 'PHEO' 'PN' 'PO4F' 'PP' 'SALINITY' 'SECCHI' 'SIF' 'SIGMA_T' 'SPCOND'
 'TDN' 'TDP' 'TN' 'TON' 'TP' 'TSS' 'WTEMP' 'VSS' 'DOC' 'PIP' 'FSS'
 'TURB_NTU' 'DO_SAT_P']


In [55]:
# Create the initial dictionary with parameter and its meaning
water_param_meaning = ['Chlorophyll-a', 'Dissolved inorganic nitrogen', 'Dissolved oxygen', 
                    'Dissolved organic nitrogen', 
                    'Dissolved organic phosphorus',
                    'Light attenuation',
                    'Ammonium (filtered)',
                    'Nitrite + nitrate (filtered)',
                    'Nitrite ( filtered)', 
                    'Nitrite ( filtered)',
                    'Particulate organic carbon',
                    'pH corrected for temperature 25 C',
                    'pheophytin',
                    'Particulate Organic Nitrogen and Particulate Nitrogen', 
                    'Orthophosphorus (filtered)', 
                    'Particulate phosphorus',
                    'Salinity', 
                    'Secchi disk depth',
                    'Silica (filtered)', 
                    'Water density',
                    'Specific conductance At 25 C',
                    'Total dissolved nitrogen',
                    'Total dissolved phosphorus', 
                    'Total nitrogen',
                    'Total organic nitrogen',
                    'Total phosphorus',
                    'Total suspended solids', 
                    'Water temperature',
                    'Volitile Suspended Solids',
                    'Dissolved organic carbon',
                    'None',
                    'Particulate inorganic phosphorus',
                    'Fixed suspended solids',
                    'Turbidity: nephelometric method',
                    'Dissolved oxygen saturation using probe units']

In [56]:
water_param_dict = param_dict(water,water_param_meaning)

We can now remove the units column.

In [58]:
water = water.drop(columns='ReportedUnits')

We will also remove the `Qualifier` column, for now.

In [63]:
water  = water.drop(columns='Qualifier')

# Turn parameters into columns

## Common functions

First, let's see what columns are common to all for cleaned dataframes and which are unique. Sice we will want to run this step a few times, we will define a function.

In [57]:
def parameter_to_columns(dataframe,columns_to_group):
    print("Original dataframe shape: ", dataframe.shape)
    # Reset index to use row numbers as the index
    df_reset = dataframe.reset_index(drop=True)

    # Pivot the DataFrame while preserving non-pivoted columns
    df_pivot = df_reset.pivot_table(index=df_reset.index, columns='ReportedParameter', values='ReportedValue', aggfunc='first')

    # Combine pivoted result with the original DataFrame columns not involved in the pivot
    df_pivot_combined = df_pivot.join(df_reset,how='outer').drop(columns=['ReportedParameter', 'ReportedValue'])
    
    # Create a copy of the DataFrame for processing
    df_processed = df_pivot_combined.copy()

    # Create a unique identifier for each group based on the columns to match
    df_processed['UniqueID'] = df_processed[columns_to_group].astype(str).agg('-'.join, axis=1)

    # Group by the unique identifier
    df_combined = df_processed.groupby('UniqueID', as_index=False).first()

    # Drop the UniqueID column and remove duplicates
    df_clean = df_combined.drop(columns='UniqueID').drop_duplicates()

    print("New dataframe shape: ", df_clean.shape)

    return df_clean
    

## Tiadal Benthic

Let's find which columns are in all four tidal benthic datasets.

In [131]:
common_columns = list(set(sediment_and_water_columns) & set(biomass_columns) & set(taxonomic_columns)-set(['ReportedParameter', 'ReportedValue']))

Let's apply the function!

In [132]:
sediment_clean = parameter_to_columns(sediment,common_columns)

Original dataframe shape:  (4695, 19)
New dataframe shape:  (893, 27)


In [133]:
waterTidalBenthic_clean = parameter_to_columns(waterTidalBenthic,common_columns)

Original dataframe shape:  (8427, 20)
New dataframe shape:  (892, 24)


In [134]:
biomass_clean = parameter_to_columns(biomass, common_columns)

Original dataframe shape:  (26904, 19)
New dataframe shape:  (854, 143)


In [135]:
taxonomic_clean = parameter_to_columns(taxonomic, common_columns)

Original dataframe shape:  (26398, 25)
New dataframe shape:  (826, 432)


### More dictionary creation
Now we can do a bit more work with the dictionaries. 


#### Biomass
Let's find a good threshold for missing data vs number of parameters.

In [136]:
# Dictionary to store columns meeting each threshold
columns_meeting_thresholds = {}

# Iterate through 10% intervals from 10% to 100%
for i in range(1, 11):
    threshold_percent = i * 10
    threshold = len(biomass_clean) * threshold_percent / 100

    # Determine the columns that meet this threshold
    columns_with_threshold_values = biomass_clean.columns[biomass_clean.count() >= threshold]

    # Filter the DataFrame to include only these columns
    filtered_df = biomass_clean[columns_with_threshold_values]

    # Determine new parameters
    biomass_new_parameters = list(set(filtered_df.columns) - set(common_columns) - set(['SampleReplicate', 'BiologicalEventId']))

    # Store the results
    columns_meeting_thresholds[f'{threshold_percent}%'] = biomass_new_parameters

    # Display the results
    print(f"Number of parameters above {threshold_percent}%: {len(biomass_new_parameters)}")
    print(f"Parameters: {biomass_new_parameters}")

Number of parameters above 10%: 121
Parameters: ['TAXA_RICH_R', 'PCT_DOM1', 'SW', 'PCT_SIMULIIDAE', 'PCT_DOM2_R', 'PCT_DOM3_R', 'PCT_PREDATOR', 'PCT_SHREDDER_R', 'PCT_PS_O_ABUND', 'PCT_EPHEMEROPTERA_R', 'PLECOPTERA_TAXA_CNT', 'FBI_R', 'PCT_EPT_R', 'PLECOPTERA_TAXA_CNT_R', 'TRICHOPTERA_TAXA_COUNT_NO_HYDR', 'DIPTERA_TAXA_CNT_R', 'PCT_CHIRONOMIDAE', 'MARGALEFS_R', 'TRICHOPTERA_TAXA_COUNT_NO_R', 'EPHEMEROPTERA_TAXA_CNT_R', 'BECK_R', 'SCRAPER_TAXA_CNT_R', 'PCT_PI_BIO', 'TRICHOPTERA_TAXA_CNT_R', 'PCT_EPT_TAXA_RICH_R', 'GOLD', 'PCT_SWIMMER_R', 'PCT_TOLERANT_R', 'SIMPSON_DIVERSITY', 'PCT_PI_O_ABUND', 'PCT_TRICHOPTERA', 'SENSITIVE_TAXA_COUNT', 'FSW', 'ASPT_MOD_R', 'TOTAL_SCORE', 'PCT_FILTERERS_R', 'RATIO_SC_TO_CF', 'PCT_CLING', 'TAXA_RICH', 'PCT_LIMESTONE_R', 'PCT_BURROWER', 'PCT_NET_CADDISFLY_R', 'DIPTERA_TAXA_CNT', 'NON_INSECT_TAXA_CNT', 'EPHEMEROPTERA_TAXA_CNT', 'EPT_TAXA_ABUND_R', 'PCT_LIMESTONE', 'SIMPSON_DIVERSITY_R', 'PCT_CHIRONOMIDAE_R', 'TOLERANT_TAXA_COUNT', 'PCT_CLINGER_TAXA', 'PCT_C

Above 30% with 15 parameters looks reasonable. Let's work on creating a dictionary. 

In [137]:
threshold_percent = 30
threshold = len(biomass_clean) * threshold_percent / 100

columns_with_threshold_values = biomass_clean.columns[biomass_clean.count() >= threshold]

filtered_df = biomass_clean[columns_with_threshold_values]

# Determine new parameters
biomass_new_parameters = list(set(filtered_df.columns) - set(common_columns) - set(['SampleReplicate', 'BiologicalEventId']))

# Create the dictionary with the specified values
biomass_param_dict = {}
for param in biomass_new_parameters:
    if 'PCT' in param:
        biomass_param_dict[param] = {'Meaning': "", 'Units': 'PCT'}
    else:
        biomass_param_dict[param] = {'Meaning': "", 'Units': ""}

print(biomass_param_dict)

{'PCT_DEPO': {'Meaning': '', 'Units': 'PCT'}, 'TOT_BIOMASS_G': {'Meaning': '', 'Units': ''}, 'SW': {'Meaning': '', 'Units': ''}, 'TOLERANCE': {'Meaning': '', 'Units': ''}, 'PCT_PS_O_ABUND': {'Meaning': '', 'Units': 'PCT'}, 'PCT_TANYPODINI': {'Meaning': '', 'Units': 'PCT'}, 'PCT_PI_BIO': {'Meaning': '', 'Units': 'PCT'}, 'PCT_PI_F_ABUND': {'Meaning': '', 'Units': 'PCT'}, 'TOT_ABUND': {'Meaning': '', 'Units': ''}, 'PCT_PI_O_ABUND': {'Meaning': '', 'Units': 'PCT'}, 'PCT_PS_ABUND': {'Meaning': '', 'Units': 'PCT'}, 'PCT_PI_ABUND': {'Meaning': '', 'Units': 'PCT'}, 'TOTAL_SCORE': {'Meaning': '', 'Units': ''}, 'PCT_CARN_OMN': {'Meaning': '', 'Units': 'PCT'}, 'PCT_PS_BIO': {'Meaning': '', 'Units': 'PCT'}}


We are going to cheat a bit an read in my previous attempt at a dictionary, which are the 10 parameters that have data for at least 60% of rows.

Of course, once that file is replaced, this step will be redundant.

In [138]:
# Read the existing dictionary from a JSON file
with open('../../data/plankton-patrolCSVs/plank_CBPparam_dict.json', 'r') as file:
    previous_dict = json.load(file)

# Update the dictionary with the new data
for key, value in previous_dict.items():
    if key in biomass_param_dict:
        biomass_param_dict[key]['Meaning'] = value.get('Type', biomass_param_dict[key]['Meaning'])
        if not biomass_param_dict[key]['Units']:
            biomass_param_dict[key]['Units'] = value.get('Units', biomass_param_dict[key]['Units'])

# Print entries with empty 'Meaning'
for key, value in biomass_param_dict.items():
    if value.get('Meaning', "") == "":
        print(f"Key: {key}, Value: {value}")

Key: TOLERANCE, Value: {'Meaning': '', 'Units': ''}
Key: PCT_PS_O_ABUND, Value: {'Meaning': '', 'Units': 'PCT'}
Key: PCT_TANYPODINI, Value: {'Meaning': '', 'Units': 'PCT'}
Key: PCT_PI_F_ABUND, Value: {'Meaning': '', 'Units': 'PCT'}
Key: PCT_PI_O_ABUND, Value: {'Meaning': '', 'Units': 'PCT'}


In [139]:
# Manually update specific entries with empty 'Meaning'
if biomass_param_dict['PCT_TANYPODINI']['Meaning'] == "":
    biomass_param_dict['PCT_TANYPODINI']['Meaning'] = "Percent Tanypodinae To Chironomidae-Tidal Benthic"

if biomass_param_dict['PCT_PI_F_ABUND']['Meaning'] == "":
    biomass_param_dict['PCT_PI_F_ABUND']['Meaning'] = "Percent Pollution Indicative Species Abundance-Tidal Fresh Benthic"

if biomass_param_dict['PCT_PS_O_ABUND']['Meaning'] == "":
    biomass_param_dict['PCT_PS_O_ABUND']['Meaning'] = "Percent Pollution Sensitive Species Abundance-Oligohaline Benthic"

if biomass_param_dict['PCT_PI_O_ABUND']['Meaning'] == "":
    biomass_param_dict['PCT_PI_O_ABUND']['Meaning'] = "Percent Pollution Indicative Species Biomass-Oligohaline Benthic "

if biomass_param_dict['TOLERANCE']['Meaning'] == "":
    biomass_param_dict['TOLERANCE']['Meaning'] = "Tidal Benthic Pollution Tolerance Index"
    biomass_param_dict['TOLERANCE']['Units'] = "None"

In [140]:

# Print entries with 'Units' set to "none"
for key, value in biomass_param_dict.items():
    if value.get('Units', "").lower() == "none":
        print(f"Key: {key}, Value: {value}")

Key: SW, Value: {'Meaning': 'Shannon Wiener Index', 'Units': 'None'}
Key: TOLERANCE, Value: {'Meaning': 'Tidal Benthic Pollution Tolerance Index', 'Units': 'None'}
Key: TOTAL_SCORE, Value: {'Meaning': 'Total Benthic Restoration Goal Score For Sample', 'Units': 'None'}


Three of these vales (SW : Shannon Wiener Index, TOTAL_SCORE : Total Benthic Restoration Goal Score For Sample, and 
TOLERANCE :Tidal Benthic Pollution Tolerance Index) are calculated scores of ecosystem health with no units. Samples with index values of 3.0 or more are
considered to have good benthic condition indicative of good habitat quality.

#### Taxonomic counts
Let's find a good threshold for missing data vs number of parameters.

In [141]:
taxonomic.columns

Index(['CBSeg2003', 'CBSeg2003Description', 'Station', 'Latitude', 'Longitude',
       'SampleType', 'FieldActivityId', 'SampleDate', 'SampleTime', 'Layer',
       'TotalDepth', 'ReportedValue', 'ReportingUnit', 'EventId', 'Source',
       'GMethod', 'TSN', 'LifeStageDescription', 'LatinName',
       'ProjectIdentifier', 'SampleVolumeUnits', 'Salzone', 'PDepth',
       'SampleVolume', 'ReportedParameter'],
      dtype='object')

In [142]:
# Dictionary to store columns meeting each threshold
columns_meeting_thresholds = {}

# Iterate through 10% intervals from 10% to 100%
for i in range(1, 11):
    threshold_percent = i * 10
    threshold = len(taxonomic_clean) * threshold_percent / 100

    # Determine the columns that meet this threshold
    columns_with_threshold_values = taxonomic_clean.columns[taxonomic_clean.count() >= threshold]

    # Filter the DataFrame to include only these columns
    filtered_df = taxonomic_clean[columns_with_threshold_values]

    # Determine new parameters
    taxonomic_new_parameters = list(set(filtered_df.columns) - set(common_columns) - set(['GMethod', 'TSN', 'LifeStageDescription', 'LatinName', 'ProjectIdentifier','EventId', 'SampleType', 'ReportingUnit']))

    # Store the results
    columns_meeting_thresholds[f'{threshold_percent}%'] = taxonomic_new_parameters

    # Display the results
    print(f"Number of parameters above {threshold_percent}%: {len(taxonomic_new_parameters)}")
    print(f"Parameters: {taxonomic_new_parameters}")

Number of parameters above 10%: 46
Parameters: ['Gemma gemma ', 'Marenzelleria viridis ', 'Glycinde solitaria ', 'Loimia medusa ', 'Clymenella torquata ', 'Leptocheirus plumulosus ', 'Podarkeopsis levifuscina ', 'Ameroculodes Complex', 'Leitoscoloplos Species', 'Eteone heteropoda ', 'Pectinaria gouldi ', 'Tellina agilis ', 'Macoma mitchelli ', 'Nemertea Species', 'Macoma baltica ', 'Rictaxis punctostriatus ', 'Spiochaetopterus costarum ', 'Streblospio benedicti ', 'Macroclymene zonalis ', 'Neanthes succinea ', 'Prionospio perkinsi ', 'Mediomastus ambiseta ', 'Acteocina canaliculata ', 'Micrura leidyi ', 'Cyathura polita ', 'Phyllodoce arenae ', 'Paraprionospio pinnata ', 'Ampelisca Juvenile', 'Ampelisca verrilli ', 'Caulleriella killariensis ', 'Phoronis Species', 'Tubificidae Immature Without Cap. Chaete', 'Edotea triloba ', 'Stylochus ellipticus ', 'Glycera americana ', 'Mulinia lateralis ', 'Carinoma tremaphoros ', 'Listriella barnardi ', 'Heteromastus filiformis ', 'Nephtys picta '

## Water Quality database

We can do the same thing with water quality. There is only one (very large) csv. Let's get the columns to group by calling the API for one year of monitoring data. Some of the columns were empty, so we will intersect the monitor data columns with our water data columns.

In [59]:
monitor_data = pd.read_csv('https://datahub.chesapeakebay.net/api.CSV/WaterQuality/MonitorEvent/8-5-2023/8-6-2024/2,4,6/12,13,15,35,36,2,3,7,33,34,23,24/CBSeg2003/10,11,12,13,14,15,16,17,28,49,84')
columns_to_group = monitor_data.columns.intersection(water.columns)

In [65]:
water_clean = parameter_to_columns(water, columns_to_group)

Original dataframe shape:  (2164289, 27)
New dataframe shape:  (29236, 59)


In [66]:
water_clean['FieldActivityId'].nunique()

19799

In [67]:
water_clean.to_csv('../../data/plankton-patrolCSVs/plank_ChesapeakeBayWater_clean.csv', index=False)

# Combining Tidal Benthic Datasets

Now we combine on the common columns. The taxonomic data is so sparce, we will omit it. We will include all of the biomass data, as a lot of columns can probably be combined (ones differentiated by salinity, for example).

Let's get rid of some extra columns before merging. `BiologicalEventId` is only in BioMass, `EventId` is simply a different system for recoding the same information as `FieldActivityId`. `SampleReplicate` should not matter, but migh prevent some merging.

In [ ]:
sediment_clean = sediment_clean.drop(columns=['SampleReplicate'])
waterTidalBenthic_clean = waterTidalBenthic_clean.drop(columns=['SampleReplicate'])

In [145]:
# Merge sediment_clean and biomass_clean
tidalBenthic = pd.merge(sediment_clean, biomass_clean, how='outer', on=[col for col in sediment_clean.columns if col in biomass_clean.columns], suffixes=('', '_biomass_clean'))

# Merge the result with waterTidalBenthic_clean
tidalBenthic = pd.merge(tidalBenthic, waterTidalBenthic_clean, how='outer', on=[col for col in tidalBenthic.columns if col in waterTidalBenthic_clean.columns and not col.endswith(('_biomass_clean', '_taxonomic_clean'))], suffixes=('', '_water_clean'))

# Reset the index for better readability
tidalBenthic = tidalBenthic.reset_index(drop=True)

There are also some issues with different precisions for latitude and longitude causing lack of matching.

In [146]:
#  Define columns for matching
match_columns = list(set(sediment_clean.columns) & set(waterTidalBenthic_clean.columns) & set(biomass_clean.columns))

# Generate a composite key based on the matching columns
tidalBenthic['unique_key'] = tidalBenthic[match_columns].apply(lambda row: tuple(row.fillna('missing')), axis=1)

# Handle Latitude and Longitude with precision
# Keep the most precise value for Latitude and Longitude
tidalBenthic['Latitude'] = tidalBenthic.groupby('unique_key')['Latitude'].transform(lambda x: x.dropna().iloc[0] if not x.dropna().empty else np.nan)
tidalBenthic['Longitude'] = tidalBenthic.groupby('unique_key')['Longitude'].transform(lambda x: x.dropna().iloc[0] if not x.dropna().empty else np.nan)

# Aggregate the groups
tidalBenthic_clean = tidalBenthic.groupby('unique_key').first().reset_index()

# Drop the unique_key column from the result
tidalBenthic_clean = tidalBenthic_clean.drop(columns=['unique_key'], errors='ignore')


In [147]:
tidalBenthic_clean.to_csv('../../data/plankton-patrolCSVs/plank_ChesapeakeBayBenthic_clean.csv', index=False)

# All together!

Since `FieldActivityId` mighht be coded differently, and really is used for combing rows from the same database, let's drop it from both. We probably won't need `CBSeg2003Description`, whihch is not in the water dataset. It can be added back with a dictionary.

In [148]:
water_extra_clean = water_clean.drop(columns='FieldActivityId')
tidalBenthic_extra_clean = tidalBenthic_clean.drop(columns=['FieldActivityId','CBSeg2003Description'])

In [149]:
# Merge tidalBenthic_extra and water_extra_clean
CPBDathub = pd.merge(tidalBenthic_extra_clean, water_extra_clean, how='outer', on=[col for col in tidalBenthic_extra_clean.columns if col in water_extra_clean.columns], suffixes=('_tidal', '_water'))

# Reset the index for better readability
CPBDathub = CPBDathub.reset_index(drop=True)

# Export dictionaries

Let's make sure the dictionaries can be used later!

In [44]:
# Find them!

# Get all variables in the current notebook's namespace
all_vars = globals()

# Filter variables to find dictionaries
dict_vars = {name: obj for name, obj in all_vars.items() if isinstance(obj, dict)}

# Print dictionary variables
for name, dictionary in dict_vars.items():
    print(f"Variable name: {name}")



Variable name: _oh
Variable name: Out
Variable name: sediment_param_dict
Variable name: waterTidalBenthic_param_dict
Variable name: taxonomic_names_dict
Variable name: life_stage_dict
Variable name: group_dict
Variable name: columns_meeting_thresholds
Variable name: biomass_param_dict
Variable name: previous_dict
Variable name: value
Variable name: all_vars


In [45]:
param_dicts = {**waterTidalBenthic_param_dict, **sediment_param_dict,**biomass_param_dict}

In [46]:
# import json

# # Export to JSON file
# with open('../../data/plankton-patrolCSVs/plank_CBPparam_dict.json', 'w') as f:
#     json.dump(param_dicts, f)